In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



2024-10-03 14:37:05.463636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


In [3]:
# データの読み込み
data = pd.read_csv('../data/preprocessed_data.csv')

# 特徴量とターゲット変数の分割
X = data.drop('dengue', axis=1).values
y = data['dengue'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train.shape

(3508, 13)

In [5]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-10-03 14:37:18.192736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 88 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
2024-10-03 14:37:18.193402: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 219 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1
2024-10-03 14:37:18.210483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 88.62MiB (92930048 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-10-03 14:37:18.210659: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 79.76MiB (83637248 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-10-03 14:37:18.218387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.

In [6]:
with strategy.scope():
    # モデルの定義
    model = Sequential()

    # 入力層および隠れ層
    model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_dim=13))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.25))
    
    # 出力層（シグモイド活性化関数を使用）
    model.add(Dense(1, activation='sigmoid'))

    # モデルのコンパイル
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



In [7]:
# モデルのトレーニング
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history=model.fit(X_train, y_train, epochs=50,validation_split=0.1, batch_size=32,callbacks=[early_stopping])

Epoch 1/50


2024-10-03 14:37:25.485560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [3157,13]
	 [[{{node Placeholder/_10}}]]
2024-10-03 14:37:25.485912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [3157,13]
	 [[{{node Placeholder/_10}}]]


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:

2024-10-03 14:37:27.209598: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2024-10-03 14:37:27.209641: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:222] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2024-10-03 14:37:27.209667: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:621 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
2024-10-03 14:37:27.209711: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:1] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: Attempting to perform BLAS operation using StreamE

InternalError: Graph execution error:

Detected at node 'replica_1/sequential/dense/MatMul' defined at (most recent call last):
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/threading.py", line 995, in _bootstrap
      self._bootstrap_inner()
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
      self.run()
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step
      outputs = model.train_step(data)
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
      y_pred = self(x, training=True)
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 65, in error_handler
      return fn(*args, **kwargs)
    File "/home/gonken/anaconda3/envs/dengue/lib/python3.11/site-packages/keras/engine/training.py", line 558, in __call__
    